<img src="https://uploads-ssl.webflow.com/5eae007d92687089219c5dec/5eb47da29ced78042e9768ce_long_logo.png" align="right" border="0" width="35%" >

# Montecarlo Simulation of SP500, Apple & Microsoft

**Optimal portfolio Mean-Variance Markowitz Approach**

Import the required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from yahoofinancials import YahooFinancials

Download the stock prices

In [2]:
# Select Tickers and stock history dates
ticker = 'AAPL'
ticker2 = 'MSFT'
ticker3 = 'SPY'
freq = 'daily'
start_date = '2020-01-01'
end_date = '2020-05-01'


# Function to clean data extracts
def clean_stock_data(stock_data_list):
    new_list = []
    for rec in stock_data_list:
        if 'type' not in rec.keys():
            new_list.append(rec)
    return new_list

# Construct yahoo financials objects for data extraction
aapl_financials = YahooFinancials(ticker)
mfst_financials = YahooFinancials(ticker2)
spy_financials = YahooFinancials(ticker3)


# Clean returned stock history data and remove dividend events from price history

daily_aapl_data = clean_stock_data(aapl_financials
                                     .get_historical_price_data(start_date, end_date, freq)[ticker]['prices'])
daily_msft_data = clean_stock_data(mfst_financials
                                     .get_historical_price_data(start_date, end_date, freq)[ticker2]['prices'])
daily_spy_data = clean_stock_data(spy_financials
                                     .get_historical_price_data(start_date, end_date, freq)[ticker3]['prices'])

# Function to construct data frame based on a stock and it's market index
def build_data_frame(data_list1, data_list2, data_list3):
    data_dict = {}
    i = 0
    for list_item in data_list2:
        if 'type' not in list_item.keys():
            data_dict.update({list_item['formatted_date']: {'SPY': data_list3[i]['close'], 
                                                            'AAPL': data_list1[i]['close'],
                                                            'MSFT': data_list2[i]['close']
                                                            }})
            i += 1
    tseries = pd.to_datetime(list(data_dict.keys()))
    df = pd.DataFrame(data=list(data_dict.values()), index=tseries,
                      columns=['AAPL', 'MSFT', 'SPY']).sort_index()
    return df

build_data_frame(daily_aapl_data, daily_msft_data, daily_spy_data)

## Code from:
## https://stackoverflow.com/questions/49705047/downloading-mutliple-stocks-at-once-from-yahoo-finance-python

,AAPL,MSFT,SPY
2020-01-02,300.350006,160.619995,324.869995
2020-01-03,297.429993,158.619995,322.410004
2020-01-06,299.799988,159.029999,323.640015
2020-01-07,298.390015,157.580002,322.730011
2020-01-08,303.190002,160.089996,324.450012
...,...,...,...
2020-04-24,282.970001,174.550003,282.970001
2020-04-27,283.170013,174.050003,287.049988
2020-04-28,278.579987,169.809998,285.730011
2020-04-29,287.730011,177.429993,293.209991


<img src="https://uploads-ssl.webflow.com/5eae007d92687089219c5dec/5eb47da29ced78042e9768ce_long_logo.png" align="right" border="0" width="35%" > <br>